In [1]:
import pandas as pd
import json
import datetime


# Run this pipeline to create a new json file for the knowledge graph

Read current entities and relations

In [2]:
with open('../03_Output/00_GPT KGs/Entities.json', "r") as file:
    data = file.read()
    entities = json.loads(data)
    file.close()

In [4]:
file_path = '../03_Output/00_GPT KGs/Relations_replaced.csv'
relations = pd.read_csv(file_path, delimiter=';')


Merge the entities and relations to created kg json file

In [5]:
kg_list = []
for entity in entities:
    kg_obj = {'metadata': entity, 'knowledge graph': {}}
    records = relations.loc[relations['Subject'] == entity['Entity']]
    filtered_relations = json.loads(records.to_json(orient ='records', indent = 2))
    objects = set()
    entity_relations = []
    for rel in filtered_relations:
        obj = {}
        objects.add(rel['Object'])
        obj['Relation'] = rel['Relation']
        obj['Object'] = rel['Object']
        obj['Description'] = rel['Description']
        obj['Importance'] = rel['Importance']
        entity_relations.append(obj)
    object_lst = list(objects)
    relation_obj = {entity['Entity']: entity_relations}
    kg_obj['knowledge graph']['entities'] = object_lst
    kg_obj['knowledge graph']['relations'] = relation_obj

    kg_list.append(kg_obj)


Update the current kg and add in archive for record

In [41]:
timestamp = '{:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now())
kg_json = json.dumps(kg_list, indent = 2)

with open('../03_Output/01_Auto KGs/00_Current Versions/knowledge_graph.json', 'w') as file:
    file.write(kg_json)
    file.close()

with open('../03_Output/01_Auto KGs/01_Archived Versions/' + str(timestamp) + '.json', 'w') as file:
    file.write(kg_json)
    file.close()

In [12]:
for item in kg_list:
    name = item['metadata']['Entity']
    json_ = json.dumps(item, indent = 2)
    with open('../03_Output/01_Auto KGs/00_Current Versions/' + name + '.json', 'w') as file:
        file.write(json_)
        file.close()
